# Markov

This is a Markov chain Python implementation. The main objective was simplicity instead of performance. This was accomplished by using adjacency lists rather than a transition matrix. The implementation is done with two classes, the WeightMap class to handle the weights and the MarkovNode to build the actual graph.

## WeightMap

This class only holds some keys with weights and enables random weighted choices.

In [1]:
from collections import defaultdict
from random import choices

class WeightMap(defaultdict):
    '''Keys with respective weights. Weight type must be acceptable for random.choices.'''
    
    def __init__(self, *args):
        #initialisation possibilities: https://docs.python.org/3/library/stdtypes.html#dict
        #not included children have zero probability to be drawn
        super().__init__(int, *args)
    
    def __call__(self):
        '''Returns a random weighted key.'''
        return choices(list(self.keys()), list(self.values()))[0]

In [2]:
#basic usage
from collections import Counter

wmp = WeightMap({'a':1, 'b':2})

cnt = Counter()
for _ in range(100000):
    cnt.update(wmp())
wmp['b']/wmp['a'], cnt['b']/cnt['a'] #should be nearly equal

(2.0, 1.974685427016093)

## MarkovNode

The WeightMap handles the outgoing connections of a node; MarkovNode then extends it to a complete node of the graph (WeightMap + data of node/state + utilities for training and propagation through graph).

In [3]:
class MarkovNode(WeightMap):
    '''Node to construct a Markov chain out of.'''
    
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def __hash__(self):
        return hash(self.data)
    
    def find(nodes, data, add=False):
        '''Returns the first node from the nodes with the given data. Automatic addition to the list is possible.'''
        #find node by data attribute in node list, https://stackoverflow.com/a/7125547
        node = next((node for node in nodes if node.data==data), None)
        
        #if it doesn't exist yet, create and add it or scream
        if node == None:
            if not add:
                raise KeyError
            node = MarkovNode(data)
            nodes += [node]
        
        return node
    
    def inc(self, nodes, childData, increment=1):
        '''Increments the count for the child node with the given data. If it doesn't exist yet, it is created and added.'''
        childNode = MarkovNode.find(nodes, childData, True)
        self[childNode] += increment
        return childNode
    
    def __repr__(self):
        return '{' + repr(self.data) + '| ' + ', '.join([repr(key.data) + ':' + repr(self[key]) for key in self.keys()]) + '}'
    
    #propagates through the graph, from node to node, and returns their data
    #(generator is more compact than full iterator)
    def __iter__(self):
        node = self
        while node: #continue up to node without children
            yield node.data
            node = node()
        yield node.data #don't forget last node

In [4]:
#basic usage
nodes = [MarkovNode('a'), MarkovNode('b')]
nodes[0].inc(nodes, 'b')
nodes

[{'a'| 'b':1}, {'b'| }]

## Tests

Construct a Markov chain from a training string, output it and generate an output. As start a node with an empty string is use;, as end a node with a line break (and no children) is used.

In [5]:
training = 'abababbbabababbbababaaaababababababa\n'
nodes = [MarkovNode('')]

#training
node = nodes[0]
for c in training:
    node = node.inc(nodes, c)

#propagation
print(nodes)
for c in nodes[0]:
    print(c, end='')

[{''| 'a':1}, {'a'| 'b':14, 'a':3, '\n':1}, {'b'| 'a':14, 'b':4}, {'\n'| }]
abbbaababababaababababbabbabababa


### Names, letter-wise

Same as above but with names. Result will be nothing better than random letters.

In [6]:
from requests import get

#get '\n' terminated names
names = [name+'\n' for name in get('https://raw.githubusercontent.com/dominictarr/random-name/master/first-names.txt').text.split('\r\n')]

#training
nodes = [MarkovNode('')]
for name in names:
    node = nodes[0]               #start of name
    for c in name.upper():        #letters
        node = node.inc(nodes, c)
#print(*nodes, sep='\n\n', end='\n\n')

#propagation
for _ in range(10):
    for c in nodes[0]:
        print(c, end='')

CEYL
DICIT
MMADA
JOEBOUS
RITHA
N
JOTARIA
TE
MAILIELLBEKINA
JATTRA


### Names, grams

Same as above, but with n-grams (n preceeding letters determine next single letter). Results will now sound like names.

Start node will have n-letter-long strings as children, n-letter-long strings have single letters and the end node (`'\n'`) as children, single letters and end node have no children.

In [7]:
from more_itertools import sliding_window


n = 3

nodes = [MarkovNode('')]
for name in names:
    nodes[0].inc(nodes, name[:n].upper())  
    for gramnletter in sliding_window(name.upper(), n+1):         #for all gram+letter windows in name
        gram, letter = ''.join(gramnletter[:-1]), gramnletter[-1]
        MarkovNode.find(nodes, gram, True).inc(nodes, letter)     #find node with gram and append letter node

#print(*nodes, sep='\n\n', end='\n\n')

for _ in range(100):
    s = nodes[0]().data #starting gram
    while s[-1] != '\n':
        s += MarkovNode.find(nodes, s[-n:])().data #find last gram and append letter
    print(s, end='')

MARYLIS
BETH
PEG
WILE
BERTINE
MARGERIE-JEANORE
DARISTY
DOLORRILLA
VIDANICA
AILYNDA
LYDA
HALENE
SELIA
CRISTEFANICAELA
HELLY-ANNE
FREDIKTA
RHETTE
JANE
NISSALLIA
JOANNA
GLOREETA
MARINDA
MURINA
CORDELIA
SELANDRA
KELLA
AMITZI
THOMASIAHALISSIE
ANGILBERTINA
REN
CLARILYSA
ZSA
SANDA
GUELYNNA
DAFFIE
GLYN
KARETH
JOELYNDEE
TAMMI
SHELENNYETTE
LAURORY
HARLOT
YOVONNA
BERNY
RAH
GERTA
DELLE
ELINA
BRITTA
SILEANNA
BELLE
BILINNIS
GILBERGETTA
MADELLE
BESSY
SON
CHLO
BRANDRA
ALLE
CHERIANORRITA
KRISA
KATHRINA
SIBETHERE
MARTEN
THE
MAURETH
CHANNA
PAULIE
CHANIQUINNYA
MERIDGE
KIT
BRIT
RONELLA
ZUZANNETA
MURIAN
DEDIA
SHA
RANDEEE
DONNI
RORINE
ROBIN
EUGENE
MUFI
LUISAHELGE
MARINE
DELIZABELL
MONIE
LURALIARYROSEFA
SUZY
MARIELLE
INGARNE
MADDY
FAWNEE
KIPPY
KAYLEY
ILYSONJANI
KAYLA
SEA
PEARTIE
GRIEE
